In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg

import plotly
import plotly.graph_objects as go


from astral import LocationInfo
from astral.sun import sun


In [2]:
from custom_utils import *

In [5]:
data_path = "./../data/"
file_name = "data.csv"
data = pd.read_csv(data_path + file_name)
data = first_preprocess(data, data_path)
data.head()

D:\RA_work\timeseries_data\notebooks\custom_utils.py:157: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  data = data.asfreq("H")
D:\RA_work\timeseries_data\notebooks\custom_utils.py:83: FutureWarning: Series.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  data[column] = data[column].interpolate(method="linear", limit_direction="both")


,space_heating,hot_water,sockets,lighting,bld_engcons,car_chargers,weekend,bank_holiday,day_of_month,year,...,sine_month,cos_month,sine_day_of_week,cos_day_of_week,sine_forecast_winddirection,cos_forecast_winddirection,sine_sunrise_hour,cos_sunrise_hour,sine_sunset_hour,cos_sunset_hour
datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-01 00:00:00+00:00,3.425193,0.000000,0.049433,0.154873,3.629499,0.0,0,1,1,2021,...,0.5,0.866025,-0.974928,-0.222521,0.000000,1.00000,0.866025,-0.5,0.866025,-0.5
2021-01-01 01:00:00+00:00,4.012907,0.000000,0.047753,0.154846,4.215506,0.0,0,1,1,2021,...,0.5,0.866025,-0.974928,-0.222521,0.000000,1.00000,0.866025,-0.5,0.866025,-0.5
2021-01-01 02:00:00+00:00,5.342417,0.444782,0.051869,0.154997,5.994066,0.0,0,1,1,2021,...,0.5,0.866025,-0.974928,-0.222521,0.000000,1.00000,0.866025,-0.5,0.866025,-0.5
2021-01-01 03:00:00+00:00,5.756598,0.000000,0.043028,0.154626,5.954252,0.0,0,1,1,2021,...,0.5,0.866025,-0.974928,-0.222521,-0.382683,0.92388,0.866025,-0.5,0.866025,-0.5
2021-01-01 04:00:00+00:00,5.415492,0.000000,0.050306,0.154498,5.620296,0.0,0,1,1,2021,...,0.5,0.866025,-0.974928,-0.222521,-0.382683,0.92388,0.866025,-0.5,0.866025,-0.5


In [6]:
data.columns

Index(['space_heating', 'hot_water', 'sockets', 'lighting', 'bld_engcons',
       'car_chargers', 'weekend', 'bank_holiday', 'day_of_month', 'year',
       'forecast_temperature', 'forecast_feelslike', 'forecast_weathertype',
       'forecast_windspeed', 'forecast_uvindex',
       'forecast_precipitationprobability', 'forecast_visibility',
       'week_of_year', 'daylight_hours', 'is_daylight', 'sine_hour',
       'cos_hour', 'sine_month', 'cos_month', 'sine_day_of_week',
       'cos_day_of_week', 'sine_forecast_winddirection',
       'cos_forecast_winddirection', 'sine_sunrise_hour', 'cos_sunrise_hour',
       'sine_sunset_hour', 'cos_sunset_hour'],
      dtype='object')

#### Data preparation

In [7]:
exo_columns = ['forecast_temperature', 'forecast_feelslike', 'forecast_weathertype',
       'forecast_windspeed', 'forecast_uvindex',
       'forecast_precipitationprobability', 'forecast_visibility',
       'week_of_year', 'daylight_hours', 'is_daylight', 'sine_hour',
       'cos_hour', 'sine_month', 'cos_month', 'sine_day_of_week',
       'cos_day_of_week', 'sine_forecast_winddirection',
       'cos_forecast_winddirection', 'sine_sunrise_hour', 'cos_sunrise_hour',
       'sine_sunset_hour', 'cos_sunset_hour']
endo_columns = ['weekend', 'bank_holiday', 'day_of_month', 'year']
target_column = ["y"]
column_name = exo_columns + endo_columns  + target_column # + ["space_heating"]
test_date = '2023-04-01'

# transform the data
sh_data = data.copy()
sh_data = sh_data.rename(columns={"space_heating": "y"})
# sh_data["y"] = sh_data[column_name[0]].shift(1)
# sh_data = sh_data.dropna(axis=0)
sh_data_scaler, sh_transformed_data = create_std_scaler(sh_data, column_name[:-1])
sh_target_scaler, sh_transformed_target_data = create_std_scaler(sh_data, column_name[-1])

sh_transformed_data = pd.merge(sh_transformed_data, sh_transformed_target_data,
                               left_index=True,
                               right_index=True)

# # Training and testing data for comms and services
sh_train_data = sh_transformed_data[sh_transformed_data.index < test_date][column_name]
sh_test_data = sh_transformed_data[sh_transformed_data.index >= test_date][column_name]
sh_train_data.shape, sh_test_data.shape

                              columns          mean    variance  scale_factor
0                forecast_temperature  1.184456e+01   21.642635      4.652165
1                  forecast_feelslike  9.344768e+00   29.482598      5.429788
2                forecast_weathertype  6.297584e+00   23.407858      4.838167
3                  forecast_windspeed  1.508560e+01   50.910046      7.135128
4                    forecast_uvindex  1.080594e+00    2.562323      1.600726
5   forecast_precipitationprobability  1.954772e+01  732.843040     27.071074
6                 forecast_visibility  4.683942e+00    0.689986      0.830654
7                        week_of_year  2.657260e+01  226.551578     15.051630
8                      daylight_hours  0.000000e+00    0.000000      1.000000
9                         is_daylight  0.000000e+00    0.000000      1.000000
10                          sine_hour -1.845302e-17    0.500000      0.707107
11                           cos_hour -5.549425e-17    0.500000 

((19680, 27), (6600, 27))

#### Modelling

In [19]:
params = {
    'n_estimators': 150,
    'criterion':  'absolute_error',
    'max_depth': 5,
    'max_features': "sqrt",
    'random_state': 994564,
    'verbose': 0
}
forecaster = ForecasterAutoreg(regressor = RandomForestRegressor(**params),
                               lags = 24)

In [ ]:
metric, predictions = backtesting_forecaster(
    forecaster         = forecaster,
    y                  = sh_train_data['y'],
    exog               = sh_train_data[exo_columns + endo_columns],
    steps              = 24,
    metric             = 'mean_absolute_error',
    initial_train_size = sh_train_data[sh_train_data.index < '2023-01-01'].shape[0],
    refit              = True,
    n_jobs             = 'auto',
    verbose            = False,
    show_progress      = True
)

print(f"Backtest error: {metric:.2f}")
predictions.head()

C:\Users\akson\anaconda3\envs\ra_env\Lib\site-packages\skforecast\model_selection\model_selection.py:474: LongTrainingWarning: The forecaster will be fit 90 times. This can take substantial amounts of time. If not feasible, try with `refit = False`.
 
 You can suppress this warning using: warnings.simplefilter('ignore', category=LongTrainingWarning)
  warnings.warn(


  0%|          | 0/90 [00:00<?, ?it/s]

In [ ]:
metric

In [11]:
sh_validation = sh_train_data[sh_train_data.index >= "2023-01-01"]
sh_validation = sh_validation[["y"]]
sh_validation = pd.merge(sh_validation, predictions, left_index=True, right_index=True)

sh_validation.head()

,y,pred
datetime,,
2023-01-01 00:00:00+00:00,2.282837,-0.009253
2023-01-01 01:00:00+00:00,1.877998,0.044091
2023-01-01 02:00:00+00:00,0.729589,0.099400
2023-01-01 03:00:00+00:00,0.669020,0.140617
2023-01-01 04:00:00+00:00,0.563266,0.192482


In [12]:
sh_validation[["pred", "y"]] = sh_target_scaler.inverse_transform(sh_validation[["pred", "y"]])
sh_validation.head()

,y,pred
datetime,,
2023-01-01 00:00:00+00:00,3.631463,0.910539
2023-01-01 01:00:00+00:00,3.150882,0.973863
2023-01-01 02:00:00+00:00,1.787612,1.039520
2023-01-01 03:00:00+00:00,1.715712,1.088448
2023-01-01 04:00:00+00:00,1.590172,1.150017


In [13]:
cal_metrics(sh_validation["y"].to_numpy(), sh_validation["pred"].to_numpy())

,MAE,MAPE,R2_Score,SMAPE,nRMSE,RMSE,MASE
0,0.900013,82.63,0.194382,53.89,0.195172,1.250081,-0.9


#### Hyperparameter tuning

In [14]:
lags_grid = [24, (4, 8, 16, 24)]

In [15]:
max_features = ["sqrt", "log2"]

In [16]:
def search_space(trial):
  _search_space = {
      "n_estimators": trial.suggest_int("n_estimators", 150, 250, step=50),
      "max_depth": trial.suggest_int("max_depth", 3, 10, step=1),
      "lags": trial.suggest_categorical("lags", lags_grid),
      "criterion": "absolute_error",
      "max_features": trial.suggest_categorical("max_features", max_features),
      "warm_start": True
  }
  return _search_space

In [17]:
from skforecast.model_selection import bayesian_search_forecaster

In [18]:
results_search, frozen_trial = bayesian_search_forecaster(
    forecaster=forecaster,
    y=sh_train_data["y"],
    steps=24,
    metric='mean_absolute_error',
    search_space=search_space,
    initial_train_size=sh_train_data[sh_train_data.index < "2023-01-01"].shape[0],
    refit=False,
    n_trials=20,
    return_best=True,
    n_jobs="auto",
    verbose=False,
    show_progress=True
)

  0%|          | 0/20 [00:00<?, ?it/s]

[W 2024-05-31 10:22:51,599] Trial 0 failed with parameters: {'n_estimators': 250, 'max_depth': 5, 'lags': (4, 8, 16, 24), 'max_features': 'sqrt'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\akson\anaconda3\envs\ra_env\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\akson\anaconda3\envs\ra_env\Lib\site-packages\skforecast\model_selection\model_selection.py", line 1532, in _objective
    metrics, _ = backtesting_forecaster(
                 ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\akson\anaconda3\envs\ra_env\Lib\site-packages\skforecast\model_selection\model_selection.py", line 743, in backtesting_forecaster
    metrics_values, backtest_predictions = _backtesting_forecaster(
                                           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\akson\anaconda3\envs\ra_env\Lib\site-packages\skforecast\model_se

KeyboardInterrupt: 

In [ ]:
results_search.sort_values(by=["mean_absolute_error"], ascending=False).head()